# Modelling

In [232]:
import pandas as pd
import numpy as np

In [233]:
df = pd.read_csv("data/games_with_features.csv", index_col="id")

In [234]:
df.head()

,date,home_team_score,period,postseason,season,status,visitor_team_score,home_team.id,home_team.abbreviation,home_team.conference,...,visitor_team.full_name,winner,home_team_avg_score_historical,visitor_team_avg_score_historical,home_team_id_year,visitor_team_id_year,home_team_avg_score,visitor_team_avg_score,home_avg_score_diff,visitor_avg_score_diff
id,,,,,,,,,,,,,,,,,,,,,
47179,2019-01-30,126,4,False,2018,Final,94,2,BOS,East,...,Charlotte Hornets,1,105.8,98.2,2 2018,4 2018,112.8,108.3,3.620000,-3.831707
48751,2019-02-09,112,4,False,2018,Final,123,2,BOS,East,...,LA Clippers,0,105.8,100.7,2 2018,13 2018,112.8,113.1,3.620000,0.581818
48739,2019-02-08,117,4,False,2018,Final,110,23,PHI,East,...,Denver Nuggets,1,103.3,104.2,23 2018,8 2018,117.9,108.2,8.725532,-4.670213
48740,2019-02-08,119,4,False,2018,Final,106,30,WAS,East,...,Cleveland Cavaliers,1,103.4,98.3,30 2018,6 2018,116.4,103.8,7.429268,-8.419512
48746,2019-02-08,102,4,False,2018,Final,96,26,SAC,West,...,Miami Heat,1,105.5,96.0,26 2018,16 2018,114.9,105.4,5.129268,-6.670732


### Ultra Baseline

In [242]:
# predict winner only using home_team_avg_score when playing at home
# vs visitor_team_avg_score when playing as visitor
home_should_win = df[df["home_team_avg_score"].gt(df["visitor_team_avg_score"])]
home_should_win["winner"].value_counts(normalize=True)

1    0.664878
0    0.335122
Name: winner, dtype: float64

In [243]:
s2018 = df[df["season"].eq(2018)]
# predict winner only using home_team_avg_score when playing at home
# vs visitor_team_avg_score when playing as visitor
home_should_win = s2018[s2018["home_team_avg_score"].gt(s2018["visitor_team_avg_score"])]
home_should_win["winner"].value_counts(normalize=True)

1    0.669248
0    0.330752
Name: winner, dtype: float64

> Home team wins 66% of the time when their avg score is higher   
> there is a flaw with this baseline in that it uses the average score from all games   
> that occured that season and correlates it with the winner of games that occured before   
> that average score was known

In [58]:
df[(df["home_team_avg_score"] - 5).gt(df["visitor_team_avg_score"])]["winner"].value_counts(normalize=True)

1    0.717187
0    0.282813
Name: winner, dtype: float64

> same experiment as above but only using teams that are heavy favourites (10 pt average more).  
> win percantage seems to increse 1% per 1pt advantage

In [61]:
s2020 = df[df["season"].eq(2020)]
s2019 = df[df["season"].eq(2019)]
s2018 = df[df["season"].eq(2018)]
s2017 = df[df["season"].eq(2017)]
s2016 = df[df["season"].eq(2016)]
s2015 = df[df["season"].eq(2015)]

In [206]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

### Baseline

In [270]:
lr = LogisticRegression(max_iter=500)
nb = GaussianNB()
knn = KNeighborsClassifier()
rf = RandomForestClassifier()
xgb = XGBClassifier(eval_metric="logloss", use_label_encoder=False)

models = [lr, nb, knn, rf, xgb]

In [222]:
def train_model(model, train_data, test_data):
    X_train = train_data.drop("winner", axis=1)
    y_train = train_data["winner"]
    X_test = test_data.drop("winner", axis=1)
    y_test = test_data["winner"]                     

    model = model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print(accuracy_score(y_test, y_pred))
    #print(cross_val_score(model, feats, target).mean())

In [231]:
train_data = df[df["season"].isin([2017])][["home_team_avg_score", "visitor_team_avg_score", "winner"]]
test_data = df[df["season"].isin([2018])][["home_team_avg_score", "visitor_team_avg_score", "winner"]]
for model in models:
    train_model(model, train_data, test_data)

0.6399694889397407
0.6414950419527079
0.5713196033562167
0.5812356979405034
0.5408085430968727


#### using score diff feat

In [261]:
features = ["winner", "home_team_avg_score", "visitor_team_avg_score", "home_avg_score_diff", "visitor_avg_score_diff"]
train_data = df[df["season"].isin(range(2015,2018))][features]
test_data = df[df["season"].isin([2018])][features]
for model in models:
    train_model(model, train_data, test_data)

0.6361556064073226
0.6376811594202898
0.566742944317315
0.5903890160183066
0.5881006864988558


> xgboost and random forest predict around 80% correctly when using data from the same season.  
> unfortunately when trying to predict results of future seasons the result is horrible.

##### The Big problem with this so far is that averages for the season are used to predict outcomes of games that contributed to those averages

# New Baseline

In [263]:
stats = pd.read_csv("data/stats_feats2020.csv", index_col="game_id")

In [277]:
stats = pd.read_csv("data/stats_diff2020.csv", index_col="game_id")

In [278]:
stats

,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,ft_pct,fta,ftm,oreb,pf,pts,reb,stl,turnover,winner
game_id,,,,,,,,,,,,,,,,,,,
127581,5.5,2.0,3.5,8.837500,11.0,6.0,11.103333,-3.0,0.0,-4.365833,5.0,6.0,-2.0,-0.5,12.0,1.5,-0.5,-0.5,0
127596,-1.5,-1.0,-6.0,-6.924583,7.5,1.0,-17.396528,2.0,-5.0,-9.448889,2.0,-1.0,2.0,2.0,-10.0,-4.0,0.0,1.0,0
127611,-16.5,-7.5,-21.0,-9.833838,-21.5,-10.0,-1.211919,-37.0,-14.0,-21.804747,-18.0,-13.0,-1.0,-8.0,-51.0,-22.0,-4.5,-8.0,0
127655,1.0,-7.0,2.5,-2.522348,11.0,3.5,-5.400429,8.0,-2.5,-3.364217,6.0,3.0,6.5,-2.0,1.5,9.0,2.5,-4.0,1
127706,3.0,0.5,11.0,7.287500,-3.0,0.0,-1.423889,-3.0,-0.5,-2.741667,1.5,4.0,-1.5,-5.0,3.0,9.5,-2.5,-0.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264689,2.5,-3.0,-3.0,-23.378322,-10.0,-10.0,-3.579487,-0.5,0.0,-8.148873,-1.0,-1.5,1.0,0.0,-11.5,-2.0,0.5,-2.5,1
264801,9.5,1.5,5.0,5.186869,-20.5,-4.0,9.188889,-1.0,15.5,-14.614646,-1.5,-2.0,-3.0,3.0,25.0,2.0,0.5,5.5,1
423334,-4.0,2.5,10.0,-7.229004,-9.5,-3.0,6.276714,-5.5,4.5,-5.629365,4.0,2.5,-1.0,1.0,8.5,9.0,-3.0,1.0,1


In [279]:
X = stats.drop("winner", axis=1)
y = stats["winner"]

In [280]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

In [284]:
for model in models:
    model = model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(model)
    print(accuracy_score(y_test, y_pred))
    print(cross_val_score(model, feats, target).mean())

LogisticRegression(max_iter=500)
0.7943262411347518
0.6314759635026557
GaussianNB()
0.7446808510638298
0.6240470289164284
KNeighborsClassifier()
0.7659574468085106
0.6004219891960597
RandomForestClassifier()
0.74822695035461
0.6034706977166462
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='logloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              use_label_encoder=False, validate_parameters=1, verbosity=None)
0.7730496453900709
0.6248098415724727


> this seems to good to be true   
> cross validation score seems more realistic than the accuracy score.  
> I wonder what's going on